<a href="https://colab.research.google.com/github/ShenZi-Ast/Site/blob/main/DFT_energy_Pyscf_%26_PubChem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install pyscf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 MB 11.2 MB/s eta 0:00:00


In [12]:
pip install requests

In [22]:
import requests
from pyscf import gto, dft

def get_compound_coordinates(compound_name):
    # Fetch the compound CID from PubChem
    cid_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{compound_name}/cids/JSON"
    response = requests.get(cid_url)
    response.raise_for_status()  # Raise an error for bad status codes
    cids = response.json().get('IdentifierList', {}).get('CID', [])

    if not cids:
        raise ValueError(f"No CID found for compound: {compound_name}")

    cid = cids[0]

    # Fetch the 3D conformer for the compound using its CID
    conformer_url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/record/JSON?record_type=3d"
    response = requests.get(conformer_url)
    response.raise_for_status()
    record = response.json()

    # Print the JSON response for debugging
    print("Raw JSON response:", record)

    compounds = record.get('PC_Compounds', [])

    if not compounds:
        raise ValueError(f"No compound data found for CID: {cid}")

    record = compounds[0]

    if 'coords' not in record or not record['coords']:
        raise ValueError(f"No 3D coordinates found for compound: {compound_name}")

    coords_data = record['coords'][0].get('conformers', [])
    if not coords_data:
        raise ValueError(f"No conformers found for compound: {compound_name}")

    conformer = coords_data[0]
    x_coords = conformer.get('x', [])
    y_coords = conformer.get('y', [])
    z_coords = conformer.get('z', [])

    if not (x_coords and y_coords and z_coords):
        raise ValueError(f"No 3D coordinates found in conformer for compound: {compound_name}")

    atoms = record['atoms']['element']

    # Create a string with atom symbols and their 3D coordinates
    element_symbols = [
        '', 'H', 'He',
        'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne',
        'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar',
        'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
        'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr'
    ]

    compound_coords = ""
    for i in range(len(atoms)):
        element = element_symbols[atoms[i]]
        x = x_coords[i]
        y = y_coords[i]
        z = z_coords[i]
        compound_coords += f"{element} {x} {y} {z}; "

    return compound_coords.strip('; ')

def calculate_dft(compound_name):
    # Get the atomic coordinates for the given compound name
    compound = get_compound_coordinates(compound_name)

    # Create a molecule with the given compound coordinates
    mol = gto.M(atom=compound, basis='sto-3g')

    # Set up DFT calculation
    mf = dft.RKS(mol)
    mf.xc = 'PBE'  # Exchange-correlation functional
    energy = mf.kernel()

    print(f'The DFT calculated energy for {compound_name} ({compound}) is {energy} Hartree')

    return energy

# Example usage
# Change the following line to specify your compound name
compound_name = 'Tetrahydrocannabinol'  # Example for water

energy = calculate_dft(compound_name)
print(f'The DFT energy for {compound_name} is: {energy} Hartree')


Raw JSON response: {'PC_Compounds': [{'id': {'id': {'cid': 16078}}, 'atoms': {'aid': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53], 'element': [8, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, 'bonds': {'aid1': [1, 1, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 9, 9, 10, 11, 12, 12, 12, 13, 13, 13, 14, 15, 15, 16, 16, 16, 17, 17, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23], 'aid2': [5, 11, 14, 46, 4, 5, 6, 24, 7, 9, 25, 12, 13, 8, 26, 27, 11, 14, 10, 28, 29, 10, 30, 16, 15, 31, 32, 33, 34, 35, 36, 18, 17, 37, 38, 39, 40, 18, 19, 41, 20, 42, 43, 21, 44, 45, 22, 47, 48, 23, 49, 50, 51, 52, 53], 'order': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 